In [1]:
import pandas as pd
import csv
from tqdm import tqdm

In [2]:
drug_list = []
alt_list = []
code_list = []
drug_alternative_names = []

with open('drug_list_updated_v2.csv', 'r', encoding='latin-1') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        code_list.append(row['drug'].split(':')[1].strip()[:4])
        name = row['drug'].split(':')[2].strip().lower()
        drug_list.append(name)
        temp = row['alt'].lower().split(", ")
        # if name not in temp:
        temp.append(name)
        temp = list(set(temp))
        alt_list.append(temp)

for i in range(len(drug_list)):
    temp = (drug_list[i], alt_list[i])
    drug_alternative_names.append(temp)
    
print(drug_list)
print(alt_list)
print(code_list)
print(drug_alternative_names)


['amphetamine', 'dl-methamphetamine racemic base', 'd-methamphetamine', 'lisdexamfetamine', 'methylphenidate (dl;d;l;isomers)', 'gamma hydroxybutyric acid(fda approv', 'amobarbital (schedule 2)', 'butalbital', 'pentobarbital (schedule 2)', 'secobarbital (schedule 2)', 'lysergide(d-lsd)', 'tetrahydrocannabinolsynthetic', 'cannabicyclol', 'nabilone', 'mescaline', 'bufotenine', 'psilocybin', 'eticyclidine (pce)', 'phencyclidine (pcp)', 'alphaprodine', 'anileridine', 'cocaine', 'codeine', 'buprenorphine', 'dihydrocodeine', 'oxycodone', 'hydromorphone', 'diphenoxylate', 'hydrocodone', 'hydrocodone', 'heroin', 'levorphanol', 'meperidine (pethidine)', 'methadone', 'morphine', 'normorphine', 'morphine-3-ethereal sulfate', 'opium', 'opium powdered', 'oxymorphone', 'noroxymorphone', 'alfentanil', 'remifentanil', 'sufentanil base', 'carfentanil', 'tapentadol', 'fentanyl base']
[['bennies', 'uppers', 'addys', 'amphetamine', 'vyvanse', 'dyanavel', 'mydayis', 'xelstrym', 'black beauties', 'wakey wak

In [ ]:
df = pd.read_csv("mixed_first_items.csv", quoting=csv.QUOTE_NONE, encoding='utf-8', on_bad_lines="skip", sep="|", low_memory=False)

# Get the column names
column_names = df.columns.tolist()

# Print the column names
print("Column names:", column_names)
print(df.shape)


In [ ]:
print(df.shape)

In [ ]:
# target_shops = ["Agora", "Evolution"]

# filtered_df = df[df['marketplace'].isin(target_shops)]

# # Save the filtered DataFrame to a new CSV file or modify the existing one
# filtered_df.to_csv("mixed_first_items_2_market.csv", index=False)

In [13]:
df = pd.read_csv("mixed_first_items_2_market.csv")
print(df.shape)

(62119, 15)


In [4]:
# check whether orignial drug name appears
matched_count = 0

# Select the desired columns
selected_columns = ["title", "description", "prediction"]
desired_data = df[selected_columns]

# Print the extracted data
# print(desired_data)

# Iterate through each row and check for matches
with tqdm(total=df.shape[0]) as pbar:  
    for index, row in df.iterrows():
        title = str(row["title"]).lower()
        description = str(row["description"]).lower()
        prediction = str(row["prediction"]).lower()
        
        # Check if any drug name is present in the columns
        if any(drug_name.lower() in title or drug_name.lower() in description or drug_name.lower() in prediction for drug_name in drug_list):
            df.at[index, "matched_drug"] = ";".join([d for d in drug_list if (d in title) or (d in description) or (d in prediction)])
            matched_count += 1
        pbar.update(1)

# Print the resulting DataFrame and matched count
# print("DataFrame with matched drugs:")
# print(df)

print("\nTotal matched rows:", matched_count)

# total 990000 rows

100%|██████████| 62119/62119 [00:02<00:00, 28760.68it/s]


Total matched rows: 6543


In [18]:
# # check whether any alt name appears


matched_count = 0

# Iterate through each row in the DataFrame
with tqdm(total=df.shape[0]) as pbar: 
    for index, row in df.iterrows():

        title = str(row["title"]).lower()
        description = str(row["description"]).lower()
        prediction = str(row["prediction"]).lower()
        
        # Check if any alternative drug name appears in title or description
        match_list = []
        for drug_name, alt_names in drug_alternative_names:
            drug_name = drug_name.lower()
            for alt_name in alt_names:
                alt_name = alt_name.lower()
                if alt_name in title or alt_name in description or alt_name in prediction:
                    match_list.append(drug_name)
        if len(match_list) == 0:
            # print(prediction)
            if prediction == "cannabis":
                match_list.extend(['tetrahydrocannabinolsynthetic', 'cannabicyclol', 'nabilone'])
            if prediction == "stimulants":
                match_list.extend(['Amphetamine', 'dl-methamphetamine racemic base', 'd-methamphetamine', 'lisdexamfetamine', 'methylphenidate (dl;d;l;isomers)'])
            if prediction == "psychedelics":
                match_list.extend(['lysergide(d-lsd)', 'mescaline', 'bufotenine', 'eticyclidine (pce)', 'phencyclidine (pcp)'])
            # if prediction == "miscellaneous":
            #     ['gamma hydroxybutyric acid(fda approv', 'pentobarbital (schedule 2)',  'butalbital', 'amobarbital (schedule 2)', 'secobarbital (schedule 2)', 'alphaprodine', 'anileridine']
            if prediction == "dissociatives":
                match_list.extend(['lysergide(d-lsd)', 'eticyclidine (pce)', 'phencyclidine (pcp)'])
        match_list = list(set(match_list))
        if len(match_list) > 0:
            df.at[index, "matched_drug"] = ", ".join(match_list)
            matched_count += 1

        pbar.update(1)

# Print the number of matched rows
print("Number of matched rows:", matched_count)

df.to_csv("outputt.csv", index=False)


100%|██████████| 62119/62119 [00:15<00:00, 4107.58it/s]


Number of matched rows: 45795


In [19]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("outputt.csv")

# Group the DataFrame by the "marketplace" column and count the non-empty "matched_drug" entries
marketplace_counts = df.groupby("marketplace")["matched_drug"].count()

# Calculate the total number of rows in each marketplace
marketplace_totals = df.groupby("marketplace")["marketplace"].count()

# Calculate the ratio of non-empty "matched_drug" entries to the total rows for each marketplace
ratios = marketplace_counts / marketplace_totals

# Sort the ratios in descending order
sorted_ratios = ratios.sort_values(ascending=False)

# Display the sorted ratios
print(sorted_ratios)


marketplace
Agora        0.774922
Evolution    0.706855
dtype: float64


In [ ]:
import csv

# Step 1: Read the CSV file
filename = "outputt.csv"

# Initialize dictionaries to store counts
marketplace_counts = {}
matched_drug_counts = {}

with open(filename, 'r') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        # Get the marketplace name and matched_drug value
        marketplace = row["marketplace"]
        matched_drug = row["matched_drug"]

        # If the marketplace is not already in the dictionaries, initialize it with zero counts
        if marketplace not in marketplace_counts:
            marketplace_counts[marketplace] = 0
            matched_drug_counts[marketplace] = 0

        # Increase the count for the marketplace
        marketplace_counts[marketplace] += 1

        # If matched_drug is not empty, increase the count
        if matched_drug:
            matched_drug_counts[marketplace] += 1

# Step 3: Compute the ratio for each marketplace
ratios = {}
for marketplace in marketplace_counts:
    ratios[marketplace] = matched_drug_counts[marketplace] / marketplace_counts[marketplace]

# Step 4: Sort the ratios from high to low
sorted_ratios = dict(sorted(ratios.items(), key=lambda item: item[1], reverse=True))

# Print the sorted results
for marketplace, ratio in sorted_ratios.items():
    print(f"Marketplace: {marketplace}: {ratio:.4f}")


In [ ]:
import csv

# Replace 'input.csv' with your actual CSV file name
csv_file_path = 'output.csv'

# Open the CSV file
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    # Print rows where the "matched_drug" column is not empty
    for row in csv_reader:
        print(row)